In [1]:
import warnings
warnings.filterwarnings('ignore')
##セル内複数項目表示
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [2]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [3]:
year_months = list(uselog_months["年月"].unique())
year_months
uselog_months[(uselog_months["年月"] > int("201804")) | (uselog_months["customer_id"] == "AS002855") ].head()

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

,年月,customer_id,count
0,201804,AS002855,4
2991,201805,AS002855,5
2992,201805,AS009373,4
2993,201805,AS015233,7
2994,201805,AS015315,3


In [4]:
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [5]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer.head()
print(exit_customer["exit_date"])
exit_customer["年月"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30 00:00:00
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30 00:00:00
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30 00:00:00


708     2018-03-30 00:00:00
729     2018-03-30 00:00:00
730     2018-03-30 00:00:00
770     2018-04-30 00:00:00
785     2018-04-30 00:00:00
               ...         
4049    2019-01-28 00:00:00
4050    2019-02-28 00:00:00
4068    2019-02-28 00:00:00
4086    2019-02-28 00:00:00
4099    2019-02-28 00:00:00
Name: exit_date, Length: 1350, dtype: object


In [6]:
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### ノック43：継続顧客のデータを作成しよう

In [8]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [9]:
conti_uselog.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000.0,通常,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201805,AS009373,4,3.0,XX,C01,F,2015-11-01,NaN,CA1,...,オールタイム,10500.0,通常,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
2,201805,AS015233,7,NaN,XXXXX,C01,M,2018-05-13,NaN,CA2,...,オールタイム,10500.0,入会費半額,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
3,201805,AS015315,3,6.0,XXXXX,C01,M,2015-07-01,NaN,CA1,...,オールタイム,10500.0,通常,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
4,201805,AS015739,5,7.0,XXXXX,C03,M,2017-06-01,NaN,CA1,...,ナイト,6000.0,通常,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0


In [10]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,TS707101,6,3.0,XXXX,C03,M,2017-05-01,NaN,CA1,...,ナイト,6000.0,通常,5.583333,6.0,9.0,3.0,1.0,2019-04-30,23.0
1,201902,HI777712,4,6.0,XXXXX,C01,M,2016-12-01,NaN,CA1,...,オールタイム,10500.0,通常,5.666667,6.0,9.0,3.0,1.0,2019-04-30,28.0
2,201812,PL787344,5,3.0,XXXX,C02,F,2015-07-01,NaN,CA1,...,デイタイム,7500.0,通常,3.333333,3.0,6.0,1.0,1.0,2019-04-30,45.0
3,201902,OA751110,3,6.0,XXX,C02,F,2015-11-01,NaN,CA1,...,デイタイム,7500.0,通常,5.000000,5.0,6.0,3.0,1.0,2019-04-30,41.0
4,201806,AS622023,3,3.0,XXX,C01,M,2016-06-01,NaN,CA1,...,オールタイム,10500.0,通常,5.000000,5.0,9.0,3.0,1.0,2019-04-30,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27417,201812,HD256988,7,6.0,XXXXX,C02,F,2018-07-09,NaN,CA1,...,デイタイム,7500.0,通常,7.888889,8.0,9.0,6.0,1.0,2019-04-30,9.0
27418,201901,AS625883,6,5.0,XXXX,C02,F,2015-05-01,NaN,CA1,...,デイタイム,7500.0,通常,4.750000,5.0,6.0,2.0,1.0,2019-04-30,47.0
27419,201805,HD095957,5,4.0,XXXX,C03,M,2016-03-01,NaN,CA1,...,ナイト,6000.0,通常,5.000000,5.0,8.0,2.0,1.0,2019-04-30,37.0
27420,201901,GD535028,10,5.0,XXXXXX,C03,M,2018-09-12,NaN,CA1,...,ナイト,6000.0,通常,8.428571,9.0,11.0,5.0,1.0,2019-04-30,7.0


In [11]:
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")

In [12]:
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,TS707101,6,3.0,XXXX,C03,M,2017-05-01,NaN,CA1,...,ナイト,6000.0,通常,5.583333,6.0,9.0,3.0,1.0,2019-04-30,23.0
1,201902,HI777712,4,6.0,XXXXX,C01,M,2016-12-01,NaN,CA1,...,オールタイム,10500.0,通常,5.666667,6.0,9.0,3.0,1.0,2019-04-30,28.0
2,201812,PL787344,5,3.0,XXXX,C02,F,2015-07-01,NaN,CA1,...,デイタイム,7500.0,通常,3.333333,3.0,6.0,1.0,1.0,2019-04-30,45.0
3,201902,OA751110,3,6.0,XXX,C02,F,2015-11-01,NaN,CA1,...,デイタイム,7500.0,通常,5.000000,5.0,6.0,3.0,1.0,2019-04-30,41.0
4,201806,AS622023,3,3.0,XXX,C01,M,2016-06-01,NaN,CA1,...,オールタイム,10500.0,通常,5.000000,5.0,9.0,3.0,1.0,2019-04-30,34.0


In [13]:
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201810,TS707101,6,3.0,XXXX,C03,M,2017-05-01,NaN,CA1,...,6000.0,通常,5.583333,6.0,9.0,3.0,1.0,2019-04-30,23.0,NaN
1,201902,HI777712,4,6.0,XXXXX,C01,M,2016-12-01,NaN,CA1,...,10500.0,通常,5.666667,6.0,9.0,3.0,1.0,2019-04-30,28.0,NaN
2,201812,PL787344,5,3.0,XXXX,C02,F,2015-07-01,NaN,CA1,...,7500.0,通常,3.333333,3.0,6.0,1.0,1.0,2019-04-30,45.0,NaN
3,201902,OA751110,3,6.0,XXX,C02,F,2015-11-01,NaN,CA1,...,7500.0,通常,5.000000,5.0,6.0,3.0,1.0,2019-04-30,41.0,NaN
4,201806,AS622023,3,3.0,XXX,C01,M,2016-06-01,NaN,CA1,...,10500.0,通常,5.000000,5.0,9.0,3.0,1.0,2019-04-30,34.0,NaN


In [14]:
predict_data[["membership_period","exit_date","routine_flg"]].mean()

membership_period    19.423213
routine_flg           0.857577
dtype: float64

### ノック44：予測する月の在籍期間を作成しよう

In [15]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201810,TS707101,6,3.0,XXXX,C03,M,2017-05-01,NaN,CA1,...,5.583333,6.0,9.0,3.0,1.0,2019-04-30,23.0,NaN,17,2018-10-01
1,201902,HI777712,4,6.0,XXXXX,C01,M,2016-12-01,NaN,CA1,...,5.666667,6.0,9.0,3.0,1.0,2019-04-30,28.0,NaN,26,2019-02-01
2,201812,PL787344,5,3.0,XXXX,C02,F,2015-07-01,NaN,CA1,...,3.333333,3.0,6.0,1.0,1.0,2019-04-30,45.0,NaN,41,2018-12-01
3,201902,OA751110,3,6.0,XXX,C02,F,2015-11-01,NaN,CA1,...,5.000000,5.0,6.0,3.0,1.0,2019-04-30,41.0,NaN,39,2019-02-01
4,201806,AS622023,3,3.0,XXX,C01,M,2016-06-01,NaN,CA1,...,5.000000,5.0,9.0,3.0,1.0,2019-04-30,34.0,NaN,24,2018-06-01


### ノック45：欠損値を除去しよう

In [16]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               249
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [17]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2645
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2645
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

In [18]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()
predict_data.shape

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,ナイト,M,3.0,1.0,17,0.0
1,通常,オールタイム,M,6.0,1.0,26,0.0
2,通常,デイタイム,F,3.0,1.0,41,0.0
3,通常,デイタイム,F,6.0,1.0,39,0.0
4,通常,オールタイム,M,3.0,1.0,24,0.0


(3697, 7)

In [19]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,3.0,1.0,17,0.0,0,0,1,0,0,1,0,1
1,6.0,1.0,26,0.0,0,0,1,1,0,0,0,1
2,3.0,1.0,41,0.0,0,0,1,0,1,0,1,0
3,6.0,1.0,39,0.0,0,0,1,0,1,0,1,0
4,3.0,1.0,24,0.0,0,0,1,1,0,0,0,1


In [20]:
#del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]

In [21]:
predict_data.drop(["campaign_name_通常"],axis=1,inplace=True)

In [22]:
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,3.0,1.0,17,0.0,0,0,0,0,0
1,6.0,1.0,26,0.0,0,0,1,0,0
2,3.0,1.0,41,0.0,0,0,0,1,1
3,6.0,1.0,39,0.0,0,0,0,1,1
4,3.0,1.0,24,0.0,0,0,1,0,0


### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [23]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

In [24]:
exit.head()
exit.shape
conti.head()
conti.shape

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
2842,3.0,0.0,2,1.0,0,0,1,0,0
2843,3.0,1.0,6,1.0,0,0,1,0,0
2844,3.0,0.0,11,1.0,0,0,1,0,0
2845,3.0,0.0,3,1.0,0,0,1,0,0
2846,4.0,0.0,8,1.0,0,0,0,0,1


(1052, 9)

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
619,7.0,1.0,41,0.0,0,0,0,1,1
110,6.0,1.0,15,0.0,0,0,0,1,0
2327,7.0,1.0,8,0.0,1,0,0,0,1
1588,8.0,1.0,14,0.0,0,0,1,0,1
845,7.0,1.0,29,0.0,0,0,1,0,1


(1052, 9)

In [25]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]

In [26]:
X
y

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,3.0,0.0,2,1.0,0,0,1,0,0
1,3.0,1.0,6,1.0,0,0,1,0,0
2,3.0,0.0,11,1.0,0,0,1,0,0
3,3.0,0.0,3,1.0,0,0,1,0,0
4,4.0,0.0,8,1.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
2099,2.0,1.0,27,0.0,0,0,0,1,1
2100,4.0,1.0,21,0.0,0,0,0,1,1
2101,5.0,1.0,10,0.0,1,0,0,0,0
2102,6.0,1.0,6,0.0,0,1,1,0,1


0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2099    0.0
2100    0.0
2101    0.0
2102    0.0
2103    0.0
Name: is_deleted, Length: 2104, dtype: float64

In [27]:
del X["is_deleted"]

In [28]:
X_train, X_test, y_train, y_test =sklearn.model_selection.train_test_split(X,y)

In [29]:
X_train.shape
y_train.shape

(1578, 8)

(1578,)

In [30]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

DecisionTreeClassifier(random_state=0)

[1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1.
 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0.
 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1.
 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1.

In [31]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head(10)

,y_test,y_pred
413,1.0,1.0
1169,0.0,0.0
292,1.0,1.0
1882,0.0,0.0
255,1.0,1.0
1467,0.0,0.0
363,1.0,1.0
1338,0.0,0.0
2097,0.0,0.0
405,1.0,1.0


### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

In [32]:
aaa = results_test.loc[results_test["y_test"]==results_test["y_pred"]]
aaa

,y_test,y_pred
413,1.0,1.0
1169,0.0,0.0
292,1.0,1.0
1882,0.0,0.0
255,1.0,1.0
...,...,...
1530,0.0,0.0
455,1.0,1.0
1376,0.0,0.0
928,1.0,1.0


In [33]:
correct = len(aaa)
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.9125475285171103


In [34]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9125475285171103
0.9784537389100126


In [35]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5) #５層までに
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

DecisionTreeClassifier(max_depth=5, random_state=0)

0.9201520912547528
0.9340937896070975


### ノック49：モデルに寄与している変数を確認しよう

In [36]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.322705
1,routine_flg,0.117584
2,period,0.553071
3,campaign_name_入会費半額,0.004128
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.002513
7,gender_F,0.000000


### ノック50：顧客の退会を予測しよう

In [37]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [38]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)
input_data

[3, 1, 10, 0, 1, 1, 0, 0]

In [39]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.03658537 0.96341463]]
